# PROJECT COMPANY ZUBER



## Introduccion 

Zuber, una startup de movilidad compartida que está ingresando al mercado de Chicago, requiere analizar los patrones de comportamiento y preferencias de sus usuarios potenciales, así como evaluar el impacto de factores externos en la demanda del servicio.

Para ello, se realizó una extracción estructurada de datos mediante consultas SQL optimizadas, seguida de procesos de limpieza y transformación (ETL) para garantizar la calidad de los datos. El análisis posterior permitió identificar tendencias clave y correlaciones significativas, generando insights basados en datos que respaldan la estrategia de penetración de mercado y la optimización de operaciones.

## Carga de datos 

De a cuerdo a los datos extraidos para el desarrollo del proyecto, exponemos el contenido y el nombre que adoptaran los archivos csv para el desarrollo de los dataframes y los analisis posteriores a estos:

`project_sql_result_01.csv`. Contiene los siguientes datos:

   -```company_name```: nombre de la empresa de taxis.

   -`trips_amount`: el número de viajes de cada compañía de taxis el 15 y 16 de noviembre de 2017.

`project_sql_result_04.csv`. Contiene los siguientes datos:

   -`dropoff_location_name`: barrios de Chicago donde finalizaron los viajes.

   -`average_trips`: el promedio de viajes que terminaron en cada barrio en noviembre de 2017.

`project_sql_result_07.csv`. Contiene datos sobre viajes desde el Loop hasta el Aeropuerto Internacional O'Hare. Recuerda, estos son los valores de campo de la tabla:

   -`start_ts`: fecha y hora de recogida.

   -`weather_conditions`: condiciones climáticas en el momento en el que comenzó el viaje.

   -`duration_seconds`: duración del viaje en segundos.


In [129]:
# Cargar todas las librerías
import pandas as pd
pd.options.display.max_columns = None
pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None)
import numpy as np
from scipy import stats as st


In [130]:
# project_sql_result_01.csv acogera el nombre de company_and_trips

company_and_trips=pd.read_csv('/Users/ASUS/OneDrive/Documentos/Tripleten/Proyecto Spring 8/moved_project_sql_result_01.csv', sep=',')

# project_sql_result_04.csv acogera el nombre de dropoff_location_avg 

dropoff_location_avg=pd.read_csv('/Users/ASUS/OneDrive/Documentos/Tripleten/Proyecto Spring 8/moved_project_sql_result_04.csv', sep=',')

# project_sql_result_07.csv acogera el nombre de start_weather

start_weather=pd.read_csv('/Users/ASUS/OneDrive/Documentos/Tripleten/Proyecto Spring 8/moved_project_sql_result_07.csv', sep=',')

## Nombres asignados a los DataFrames

Para garantizar un desarrollo estructurado y eficiente del proyecto, se asignaron nombres descriptivos a los *DataFrames* generados a partir de las consultas SQL. Esta práctica permite mantener un flujo de trabajo organizado y facilita la interpretación de los resultados en las distintas etapas del análisis. Las asignaciones realizadas son las siguientes:

- `company_and_trips`: corresponde al resultado de la consulta `project_sql_result_01`.
- `dropoff_location_avg`: corresponde al resultado de la consulta `project_sql_result_04`.
- `start_weather`: corresponde al resultado de la consulta `project_sql_result_07`.

A continuación, se procederá con el análisis exploratorio de los datos.


In [131]:
# visualizacion preliminar de los dataframes

company_and_trips.info()
sample_company_and_trips= company_and_trips.sample(n=5)

print()
print(sample_company_and_trips)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   company_name  64 non-null     object
 1   trips_amount  64 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ KB

                                    company_name  trips_amount
57                                   Metro Group            11
52  3721 - Santamaria Express, Alvaro Santamaria            14
22                          Chicago Independents          1296
44                    2092 - 61288 Sbeih company            27
39                   0118 - 42111 Godfrey S.Awir            33


In [132]:
# visualizacion preliminar de los dataframes
dropoff_location_avg.info()
sample_dropoff_location_avg= dropoff_location_avg.sample(n=5)

print()
print(sample_dropoff_location_avg)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94 entries, 0 to 93
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   dropoff_location_name  94 non-null     object 
 1   average_trips          94 non-null     float64
dtypes: float64(1), object(1)
memory usage: 1.6+ KB

   dropoff_location_name  average_trips
66       Washington Park      16.033333
21          North Center     393.266667
74               Beverly      10.500000
27              Avondale     208.233333
82         South Deering       7.500000


In [133]:
# visualizacion preliminar de los dataframes
start_weather.info()

sample_start_weather= start_weather.sample(n=5)

print()
print(sample_start_weather)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1068 entries, 0 to 1067
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   start_ts            1068 non-null   object 
 1   weather_conditions  1068 non-null   object 
 2   duration_seconds    1068 non-null   float64
dtypes: float64(1), object(2)
memory usage: 25.2+ KB

                start_ts weather_conditions  duration_seconds
261  2017-11-04 17:00:00                Bad            2640.0
448  2017-11-18 11:00:00               Good            2520.0
593  2017-11-11 12:00:00               Good            2015.0
178  2017-11-18 12:00:00                Bad            2934.0
394  2017-11-18 07:00:00                Bad            1322.0
